In [12]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import pickle
import sys
import torch
import importlib
from timeit import default_timer as t
sys.path.append("../../lib")
from metrics import levenshtein
import ocr_correction
from pytorch_decoding import seq2seq

In [13]:
folder = "../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish/CZ/"

In [14]:
files = sorted(os.listdir(folder))
len(files)

1

In [15]:
import glob

files = glob.glob(folder + '/**/*.txt', recursive=True)
len(files)

46

In [16]:
data = []
for f in tqdm(files):
    with open(f) as file:
        data.append(file.readlines())

  0%|          | 0/46 [00:00<?, ?it/s]

In [17]:
data = pd.DataFrame(data, 
                    columns = ["ocr_to_input", 
                               "ocr_aligned", 
                               "gs_aligned"])\
.assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
        ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
        gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

print(data.shape)
data.head()

(46, 3)


,ocr_to_input,ocr_aligned,gs_aligned
0,"bpl bal fmětu, a fteréj »fbo jadjomámá, mííu^e...","@@@@@bpl bal fmětu, a fteréj »fbo jadjomámá, m...","227 byl dal swětu, a kteréž »kdo zachowáwá, m..."
1,"— 404 děl se .mnich, že se Mariana následkem n...","@— 404@@ děl se .mnich, že se Mariana následke...","– 404 – děl se @mnich, že se Mariana následke..."
2,Pravidla vyučováni ve školách obecných. 29 všt...,@Pravidla vyučováni ve školách obecných. 29 vš...,Pravidla vyučování ve školách obecných. 29 vš...
3,"— 113 - „Vidíš, milá matinko, pokaždé, kdykoli...","@— 113 - „Vidíš, milá matinko, pokaždé, kdykol...","– 113 – „Vidíš, milá matinko, pokaždé, kdykol..."
4,"# 244 — lino nejen k hraběti, ale i k hraběnce...","@# 244 — lino nejen k hraběti, ale i k hraběnc...","– 244 – lino nejen k hraběti, ale i k hraběnc..."


In [18]:
data.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,46.000000,46.000000,46.000000
mean,1646.782609,1653.956522,1652.956522
std,1103.553189,1105.867522,1105.867522
min,719.000000,724.000000,723.000000
25%,1090.500000,1096.250000,1095.250000
50%,1347.000000,1353.000000,1352.000000
75%,1760.750000,1772.250000,1771.250000
max,5674.000000,5695.000000,5694.000000


In [19]:
vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))

142


In [20]:
distances = levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count    46.000000
mean      6.054172
std       9.943157
min       0.572656
25%       1.143085
50%       1.584924
75%       4.696774
max      46.070461
Name: cer, dtype: float64

In [21]:
distances = levenshtein(reference = data.gs_aligned, 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count    46.000000
mean      5.899352
std       9.562867
min       0.572656
25%       1.140351
50%       1.580996
75%       4.620071
max      44.213264
Name: cer, dtype: float64

In [22]:
data.to_pickle("../../data/cz/data/test.pkl")